In [1]:
import pandas as pd

In [2]:
data = []
with open("DiabLocalHbA1c_a.txt", "r", encoding="utf-16") as file:
    header = file.readline().strip().split("|")
    for line in file:
        line = line.strip().split("|")
        data.append([line[1], line[3], line[5], line[7]])

local_hba1c_df = pd.DataFrame(data, columns=[header[1], header[3], header[5], "hba1c"])

# adding 5000 to each value of first column
local_hba1c_df[header[1]] = local_hba1c_df[header[1]].astype(int) + 5000
#in second column, in each cell, if the string has '-', replace '-' with ' '
local_hba1c_df[header[3]] = local_hba1c_df[header[3]].str.replace("-", " ")
# sort by first and second column
local_hba1c_df = local_hba1c_df.sort_values(by=[header[1], header[5]])

In [3]:
sample_data = []
with open("SampleResults_a.txt", "r") as sample_file:
    sample_header = sample_file.readline().strip().split("|")
    for sample_line in sample_file:
        sample_line = sample_line.strip().split("|")
        if sample_line[4] == "GLYHB":
            sample_data.append(
                [sample_line[1], sample_line[2], sample_line[3], sample_line[5]]
            )

sample_df = pd.DataFrame(
    sample_data, columns=[sample_header[1], sample_header[2], sample_header[3], "hba1c"]
)
# adding 5000 to each value of first column
sample_df[sample_header[1]] = sample_df[sample_header[1]].astype(int) + 5000
# sort by first and second column
sample_df = sample_df.sort_values(by=[sample_header[1], sample_header[3]])

sample_df.columns = local_hba1c_df.columns

In [4]:
merged_df = pd.merge(
    sample_df,
    local_hba1c_df,
    on=["PtID", "HbA1cTestDt"],
    how="outer",
    suffixes=["_sample", "_local"],
)


merged_df["hba1c"] = merged_df["hba1c_sample"].combine_first(merged_df["hba1c_local"])
merged_df["Visit"] = merged_df["Visit_sample"].combine_first(merged_df["Visit_local"])


merged_df = merged_df.drop(
    columns=["hba1c_local", "hba1c_sample", "Visit_local", "Visit_sample"]
)

In [5]:
screening_data = []
with open("DiabScreening_a.txt", "r", encoding="utf-16") as screening_file:
    screening_header = screening_file.readline().strip().split("|")
    for screening_line in screening_file:
        screening_line = screening_line.strip().split("|")
        screening_data.append([
        screening_line[1],
        screening_line[5], screening_line[6],
        screening_line[22], screening_line[23]
    ])

    screening_df = pd.DataFrame(screening_data, columns=[
        screening_header[1], screening_header[5], screening_header[6],
        screening_header[22], screening_header[23]])
    #add 5000 to each value of first column
    screening_df[screening_header[1]] = screening_df[screening_header[1]].astype(int) + 5000
    #sort by first column
    screening_df = screening_df.sort_values(by=[screening_header[1]])

In [6]:
roster_data = []
with open("PtRoster_a.txt", "r", encoding="utf-16") as roster_file:
    roster_header = roster_file.readline().strip().split("|")
    for roster_line in roster_file:
        roster_line = roster_line.strip().split("|")
        roster_data.append([
        roster_line[0],
        roster_line[1], roster_line[2],
        roster_line[3], roster_line[4]
    ])

    roster_df = pd.DataFrame(roster_data, columns=[
        roster_header[0], roster_header[1], roster_header[2],
        roster_header[3], roster_header[4]])
    #add 5000 to each value of first column
    roster_df[roster_header[0]] = roster_df[roster_header[0]].astype(int) + 5000
    #sort by first column
    roster_df = roster_df.sort_values(by=[roster_header[0]])

In [7]:
merged_screening_roster_df = pd.merge(
    screening_df, roster_df, on="PtID", how="outer"
)

In [8]:
merged_df.to_csv('brown2019_hba1c.csv', index=False)
merged_screening_roster_df.to_csv('brown2019_demographics.csv', index=False)

# for redo cgm part

In [9]:
import pandas as pd

In [10]:
cgm_data = []
with open("cgm.txt", "r") as cgm_file:
    cgm_header = cgm_file.readline().strip().split("|")
    for cgm_line in cgm_file:
        cgm_line = cgm_line.strip().split("|")
        cgm_data.append(cgm_line)

    cgm_df = pd.DataFrame(cgm_data, columns=cgm_header)
    # add 5000 to each value of first column
    cgm_df[cgm_header[0]] = cgm_df[cgm_header[0]].astype(int) + 5000

In [38]:
cgm_df.head()

,PtID,Period,DataDtTm,CGM
0,5001,1. Baseline,11DEC17 23:59:25,172
1,5001,1. Baseline,12DEC17 00:04:24,170
2,5001,1. Baseline,12DEC17 00:09:24,167
3,5001,1. Baseline,12DEC17 00:14:25,163
4,5001,1. Baseline,12DEC17 00:19:25,160


In [37]:
cgm_df["DataDtTm"] = cgm_df["DataDtTm"].str.replace(":", " ", 1)

In [39]:
cgm_df["time"] = pd.to_datetime(cgm_df["DataDtTm"], format="mixed", dayfirst=True)

In [40]:
cgm_df.head()

,PtID,Period,DataDtTm,CGM,time
0,5001,1. Baseline,11DEC17 23:59:25,172,2017-12-11 23:59:25
1,5001,1. Baseline,12DEC17 00:04:24,170,2017-12-12 00:04:24
2,5001,1. Baseline,12DEC17 00:09:24,167,2017-12-12 00:09:24
3,5001,1. Baseline,12DEC17 00:14:25,163,2017-12-12 00:14:25
4,5001,1. Baseline,12DEC17 00:19:25,160,2017-12-12 00:19:25


In [41]:
first_date = cgm_df.groupby("PtID")["time"].min()
last_date = cgm_df.groupby("PtID")["time"].max()

In [42]:
date_diff = (last_date - first_date).dt.days + 1

In [43]:
# Reset index of date_diff to make 'id' a column
date_diff = date_diff.reset_index()

# Rename the 'time' column (which represents the date difference)
# to something more descriptive, like 'date_diff'
date_diff = date_diff.rename(columns={"time": "date_diff"})

# Now, merge date_diff with cgm using 'id' as the key
cgm_df = pd.merge(cgm_df, date_diff, on="PtID", how="left")

In [44]:
# Group by PtID and calculate the number of unique days
unique_days = cgm_df.groupby("PtID")["time"].apply(lambda x: x.dt.normalize().nunique())

In [45]:
# Reset index of date_diff to make 'id' a column
unique_days = unique_days.reset_index()

# Rename the 'time' column (which represents the date difference)
# to something more descriptive, like 'date_diff'
unique_days = unique_days.rename(columns={"time": "unique_days"})

# Now, merge date_diff with cgm using 'id' as the key
cgm_df = pd.merge(cgm_df, unique_days, on="PtID", how="left")

In [46]:
cgm_df.head()

,PtID,Period,DataDtTm,CGM,time,date_diff,unique_days
0,5001,1. Baseline,11DEC17 23:59:25,172,2017-12-11 23:59:25,203,204
1,5001,1. Baseline,12DEC17 00:04:24,170,2017-12-12 00:04:24,203,204
2,5001,1. Baseline,12DEC17 00:09:24,167,2017-12-12 00:09:24,203,204
3,5001,1. Baseline,12DEC17 00:14:25,163,2017-12-12 00:14:25,203,204
4,5001,1. Baseline,12DEC17 00:19:25,160,2017-12-12 00:19:25,203,204


In [47]:
# no data len<= 1 day
cgm_df["wearing_ratio"] = cgm_df["unique_days"] / cgm_df["date_diff"]
cgm_df["enough_data"] = cgm_df["wearing_ratio"] >= 0.7

In [48]:
cgm_df[cgm_df["unique_days"] - cgm_df["date_diff"] > 1]

,PtID,Period,DataDtTm,CGM,time,date_diff,unique_days,wearing_ratio,enough_data


In [49]:
cgm_df.head()

,PtID,Period,DataDtTm,CGM,time,date_diff,unique_days,wearing_ratio,enough_data
0,5001,1. Baseline,11DEC17 23:59:25,172,2017-12-11 23:59:25,203,204,1.004926,True
1,5001,1. Baseline,12DEC17 00:04:24,170,2017-12-12 00:04:24,203,204,1.004926,True
2,5001,1. Baseline,12DEC17 00:09:24,167,2017-12-12 00:09:24,203,204,1.004926,True
3,5001,1. Baseline,12DEC17 00:14:25,163,2017-12-12 00:14:25,203,204,1.004926,True
4,5001,1. Baseline,12DEC17 00:19:25,160,2017-12-12 00:19:25,203,204,1.004926,True


In [50]:
cgm_filtered = cgm_df[cgm_df["enough_data"] == True]

In [51]:
cgm_filtered = cgm_filtered.rename(columns={"PtID": "id", "CGM": "gl"})

In [52]:
# filter out rows with time or gl is na
cgm_filtered = cgm_filtered.dropna(subset=["time", "gl"])

In [53]:
cgm_filtered.to_csv("brown2019_filtered_new.csv", index=False)